In [1]:
#Import all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import re

In [5]:
# Load street data
Service_Request_filepath = 'ServiceRequest.xlsx'
df_service_request = pd.read_excel(Service_Request_filepath)

Collections_timesheet_Sample_filepath = 'Collections_timesheet_Sample.xlsx'
df_coll_timesheet = pd.read_excel(Collections_timesheet_Sample_filepath)

<h2>Inspecting Service Request Data

In [4]:
df_service_request.columns

Index(['SR_NUM', 'LAGAN_ID', 'ADDRKEY', 'PROB', 'SERVICE_GROUP',
       'SERVICE_TYPE', 'SERVICE_SUBTYPE', 'DUMPSTER_IND', 'ADDDTTM', 'RESDTTM',
       'RESCODE', 'RESFLAG', 'SR_YEAR', 'SR_MONTH', 'OUTLIER_IND', 'INTAKE',
       'RES_DESC', 'GPSX', 'GPSY', 'VALID_COORDS', 'VALID_ADDR', 'CASTNO',
       'CASTNAME', 'CASUFFIX'],
      dtype='object')

In [7]:
if df_service_request is not None:
    # Display basic information about the dataframe
    print(df_service_request.info())
    print(df_service_request.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26010 entries, 0 to 26009
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   SR_NUM           26010 non-null  int64         
 1   LAGAN_ID         25985 non-null  float64       
 2   ADDRKEY          26010 non-null  int64         
 3   PROB             26010 non-null  int64         
 4   SERVICE_GROUP    26010 non-null  object        
 5   SERVICE_TYPE     26010 non-null  object        
 6   SERVICE_SUBTYPE  26010 non-null  object        
 7   DUMPSTER_IND     26010 non-null  object        
 8   ADDDTTM          26010 non-null  datetime64[ns]
 9   RESDTTM          24581 non-null  datetime64[ns]
 10  RESCODE          24581 non-null  float64       
 11  RESFLAG          26010 non-null  object        
 12  SR_YEAR          26010 non-null  int64         
 13  SR_MONTH         26010 non-null  int64         
 14  OUTLIER_IND      26010 non-null  int64

<h2>Inspecting Collections Timesheet Sample Data

In [6]:
df_coll_timesheet.columns

Index(['BEAT_NUMBER', 'TRUCK_ID', 'SERVICE_TYPE', 'MATERIAL', 'DRIVER',
       'SERVICE_DAY', 'SERVICE_DATE', 'BREAKDOWN_IND', 'OVERTIME_IND'],
      dtype='object')

In [8]:
if df_coll_timesheet is not None:
    # Display basic information about the dataframe
    print(df_coll_timesheet.info())
    print(df_coll_timesheet.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13664 entries, 0 to 13663
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BEAT_NUMBER    13664 non-null  object        
 1   TRUCK_ID       13664 non-null  object        
 2   SERVICE_TYPE   12883 non-null  object        
 3   MATERIAL       13664 non-null  object        
 4   DRIVER         13663 non-null  object        
 5   SERVICE_DAY    13664 non-null  object        
 6   SERVICE_DATE   13664 non-null  datetime64[ns]
 7   BREAKDOWN_IND  13664 non-null  int64         
 8   OVERTIME_IND   13664 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 960.9+ KB
None
  BEAT_NUMBER TRUCK_ID    SERVICE_TYPE           MATERIAL DRIVER SERVICE_DAY  \
0           E     None  Semi-Automated  Facilities Routes     Bo  not listed   
1           E    A5717  Semi-Automated  Facilities Routes     Di  not listed   
2           E  

<h2>Datasets Overview


<b>Service Request Data:</b>
Contains information about service requests including spatial data, service types, request times, and resolutions.
Key columns: SR_NUM, SERVICE_GROUP, SERVICE_TYPE, ADDDTTM, RESDTTM, GPSX, GPSY.
    
    
<b>Collections Timesheet Data:</b>
Contains details about collection timesheets including truck IDs, service types, and service dates.
Key columns: BEAT_NUMBER, TRUCK_ID, SERVICE_TYPE, SERVICE_DATE.

    
<b>Business Rules</b>
A Service request is valid if received within 5 days post-service date.

<h2>Star Schema Components:

<h3>Fact Table

Fact_ServiceRequests: Stores metrics related to service requests.

Columns:
SR_NUM (Primary Key)
ADDRKEY
SERVICE_GROUP
SERVICE_TYPE
ADDDTTM
RESDTTM
VALID_COORDS
VALID_ADDR
GPSX
GPSY


<h3>Dimension Tables

<b>Dim_Time:</b> Stores date and time attributes.

Columns:
DateKey (Primary Key)
Date
Year
Month
Day
Weekday


<b>Dim_ServiceType:</b> Stores service type information.
Columns:
ServiceTypeKey (Primary Key)
ServiceType
ServiceSubType
Dim_Truck: Stores truck-related information from timesheet data.
Columns:
TruckID (Primary Key)
BeatNumber
Driver
ServiceDay
BreakdownInd
OvertimeInd


<h4>Relationships


Fact_ServiceRequests:
Linked to Dim_Time on ADDDTTM and RESDTTM.
Linked to Dim_ServiceType on SERVICE_TYPE.
Linked to Dim_Truck on TRUCK_ID and SERVICE_DATE from timesheet data.


<h3>Star Schema ERD


The ERD will show the central Fact_ServiceRequests table connected to Dim_Time, Dim_ServiceType, and Dim_Truck dimension tables. 

Below is the description of the diagram in crow's foot notation:

Fact_ServiceRequests (SR_NUM, ADDRKEY, SERVICE_GROUP, SERVICE_TYPE, ADDDTTM, RESDTTM, VALID_COORDS, VALID_ADDR, GPSX, GPSY)

Linked to Dim_Time on ADDDTTM and RESDTTM.
Linked to Dim_ServiceType on SERVICE_TYPE.
Linked to Dim_Truck on TRUCK_ID and SERVICE_DATE.
Dim_Time (DateKey, Date, Year, Month, Day, Weekday)

1
relationship with Fact_ServiceRequests.
Dim_ServiceType (ServiceTypeKey, ServiceType, ServiceSubType)

1
relationship with Fact_ServiceRequests.
Dim_Truck (TruckID, BeatNumber, Driver, ServiceDay, BreakdownInd, OvertimeInd)

1
relationship with Fact_ServiceRequests.